In [57]:
## Make sure all libraries are installed and they work
import datetime
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import doc2vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import re
import xgboost 
from xgboost import XGBClassifier
import pandas as pd
import logging
import numpy as np
from numpy import random
import nltk
from sklearn import naive_bayes,svm, metrics,naive_bayes,ensemble
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from nltk.corpus import stopwords
from sklearn import utils
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
%matplotlib inline

In [58]:
# function needed to label each document (part of the pre processing step for doc2vec)
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

# adding C to the classess that dont have it 
def addClass(x):
    firstWord = x[0]
    if (firstWord!='C'):
        x = "C" + str(x)
    return str(x)

#function to clean the data
def clean_text(text):
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords.words('english'))
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

# function needed to label each document (part of the pre processing step for doc2vec)
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

# function to get vector for the doc2vec model
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

# coverting all elements of a list to string
def arrayToString(x):
    return [str(i) for i in x]

def convertToString(x):
    x = str(x)
    return x

# convert array to a string
def splitString(x):
    x1 = ' '.join(x)
    return x1

def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_y, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return [metrics.accuracy_score(predictions, valid_y),predictions]

In [59]:
############################################################################################################
############################################################################################################
############################################################################################################
########################################## PRE PROCESS TEST DATA ###########################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################

In [60]:
#pre process and extraction of data
file = "./testing_docs.txt"
with open(file,'r',encoding="utf8") as theFile:

    text = theFile.readlines()
text[0:5]
textList = []
i=0
j=len(text)
while i<j:
    textList.append(text[i:i+2])
    i+=4
print(len(textList))
i=0
j=len(textList)
while i<j:
    textList[i][0]=textList[i][0][3:-1]
    textList[i][1]=textList[i][1][5:-1]
    i+=1
#textList[0:2]
print(len(textList))

# creating data frame, one column for doc and one for the text in it
cols = ['ID','TEXT']
assignTestSet = pd.DataFrame(textList[0:],columns=cols)

26610
26610


In [61]:
assignTestSet['TEXT'] = assignTestSet['TEXT'].apply(clean_text)

In [62]:
len(assignTestSet)
assignTestSet.head()

,ID,TEXT
0,te_doc_1,police royal commission western australia hear...
1,te_doc_2,northern territory government says queensland ...
2,te_doc_3,group hepatitis c sufferers infected blood tra...
3,te_doc_4,crew north korean vessel pong su appear sydney...
4,te_doc_5,new south wales supreme court told magistrate ...


In [63]:
len(assignTestSet)

26610

In [64]:
############################################################################################################
############################################################################################################
############################################################################################################
######################################### PRE PROCESS TRAIN DATA ###########################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################

In [65]:
#pre process and extraction of data
file = "./training_docs.txt"
with open(file,'r',encoding="utf8") as theFile:

    text = theFile.readlines()
text[0:5]
textList = []
i=0
j=len(text)
while i<j:
    textList.append(text[i:i+2])
    i+=4
print(len(textList))
i=0
j=len(textList)
while i<j:
    textList[i][0]=textList[i][0][3:-1]
    textList[i][1]=textList[i][1][5:-1]
    i+=1
#textList[0:2]
print(len(textList))

106445
106445


In [66]:
# creating data frame, one column for doc and one for the text in it
cols = ['ID','TEXT']
trDocs = pd.DataFrame(textList[0:],columns=cols)

In [67]:
# pre processing label
file = "./training_labels_final.txt"
with open(file,'r',encoding="utf8") as theFile:
    text = theFile.readlines()

In [68]:
# seperating out the labels 
theLabel = []
i=0
while i < len(text):
    theLabel.append(text[i][-3:-1])
    i=i+1

In [69]:
# adding labels to the dataframe
theLabel = pd.Series(theLabel)
trDocs['doc.class'] = theLabel

In [70]:
# data frame ready
trDocs.head()

,ID,TEXT,doc.class
0,tr_doc_1,Two German tourists have been found safe and w...,C1
1,tr_doc_2,ACT police have seized a rare drug during a ra...,C1
2,tr_doc_3,A 50-year-old Brisbane man has been charged wi...,C1
3,tr_doc_4,In-depth discussions are continuing to resolve...,C1
4,tr_doc_5,Homicide detectives are still questioning a ma...,C1


In [71]:
trDocs['doc.class'] = trDocs['doc.class'].apply(addClass)
# trDocs['doc.class'] = trDocs['doc.class'].convertToString(addClass)
trDocs['doc.class'].unique()

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23'], dtype=object)

In [72]:
random.seed(2174)
trDocs = trDocs.sample(frac=1).reset_index(drop=True)

In [73]:
trDocs.head()

,ID,TEXT,doc.class
0,tr_doc_51653,"Serena Williams, who underwent knee surgery in...",C12
1,tr_doc_62093,"The State Member for Barwon, Ian Slack-Smith, ...",C14
2,tr_doc_103628,Eleven miners have been killed and another 25 ...,C23
3,tr_doc_51774,World number 15 Jelena Dokic has withdrawn fro...,C12
4,tr_doc_49593,Melbourne Victory captain Kevin Muscat has lab...,C11


In [74]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

trDocs['TEXT'] = trDocs['TEXT'].apply(clean_text)

In [75]:
thelist = np.arange(1,106445)
removeRows = np.random.choice(thelist,5).tolist()
removeRows

[4102, 102089, 6525, 87597, 11603]

In [76]:
trDocs1 = trDocs.drop(removeRows)

In [77]:
trDocs.head()

,ID,TEXT,doc.class
0,tr_doc_51653,serena williams underwent knee surgery august ...,C12
1,tr_doc_62093,state member barwon ian slacksmith called redu...,C14
2,tr_doc_103628,eleven miners killed another 25 trapped two se...,C23
3,tr_doc_51774,world number 15 jelena dokic withdrawn next we...,C12
4,tr_doc_49593,melbourne victory captain kevin muscat labelle...,C11


In [78]:
trDocs1.head()
print('length of trDocs',len(trDocs1))

length of trDocs 106440


In [79]:
assignTestSet.head()

,ID,TEXT
0,te_doc_1,police royal commission western australia hear...
1,te_doc_2,northern territory government says queensland ...
2,te_doc_3,group hepatitis c sufferers infected blood tra...
3,te_doc_4,crew north korean vessel pong su appear sydney...
4,te_doc_5,new south wales supreme court told magistrate ...


In [80]:
dfx = trDocs1.append(assignTestSet)
dfx = dfx.reset_index()

In [81]:
len(dfx)

133050

In [82]:
dfx.tail()

,index,ID,TEXT,doc.class
133045,26605,te_doc_26606,new report warns average temperatures rise 6 d...,NaN
133046,26606,te_doc_26607,international report shown students smaller we...,NaN
133047,26607,te_doc_26608,chinese farmer invented survival pod hopes ado...,NaN
133048,26608,te_doc_26609,friday end mayan calendar people believe also ...,NaN
133049,26609,te_doc_26610,scientists united states offered molecularleve...,NaN


In [85]:
# splitting into test and train data set
X_train, X_test, y_train, y_test = train_test_split(dfx['TEXT'], dfx['doc.class'], test_size=0.2, shuffle = False)

In [88]:
# labeling each document part of preproces for doc2vec
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [89]:
len(all_data)

133050

# Running doc2Vec

In [ ]:
start = datetime.datetime.now()
# running model doc2vec
model_dbow = Doc2Vec(dm=0, vector_size=500, negative=5, min_count=1, alpha=0.065, min_alpha=0.045)
model_dbow.build_vocab([x for x in tqdm(all_data)])

# need to change the range to train model better
# someone try it with 100
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha
print(datetime.datetime.now() - start)

100%|██████████| 133050/133050 [00:00<00:00, 2016179.04it/s]


In [ ]:
# try getting more vectors, instead of 300 try 500 or 1000
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 500, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 500, 'Test')

In [ ]:
len(train_vectors_dbow)

# Running models

## Trying LDA using cross validation

In [ ]:
start = datetime.datetime.now()
clf = LinearDiscriminantAnalysis()
scores = cross_val_score(clf, train_vectors_dbow, y_train, cv=5)
scores

In [ ]:
print(datetime.datetime.now() - start)

## Trying SVM (radial kernal) using cross validation

In [ ]:
start = datetime.datetime.now()
clf1 = SVC(gamma='auto',kernel='radial')
scores1 = cross_val_score(clf1, train_vectors_dbow, y_train, cv=5)
scores1

In [ ]:
print(datetime.datetime.now() - start)

## Trying logistic regression using cross validation

In [ ]:
start = datetime.datetime.now()
logreg = LogisticRegression(n_jobs=1, C=1e5)
start = datetime.datetime.now()
logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
datetime.datetime.now() - start 
print('Testing accuracy LR %s' % accuracy_score(y_test, y_pred))
print(datetime.datetime.now() - start)

## Trying LDA using cross validation

In [ ]:
# trying lda
clf = LinearDiscriminantAnalysis()
clf.fit(train_vectors_dbow, y_train)
x = clf.predict(test_vectors_dbow)
print('Testing accuracy lda %s' % accuracy_score(y_test, x))
f1_score(y_test, x,average='weighted')

## Trying QDA using cross validation

In [ ]:
# trying qda
qda = QuadraticDiscriminantAnalysis()
qda.fit(train_vectors_dbow, y_train)
x1 = qda.predict(test_vectors_dbow)
print('Testing accuracy qda %s' % accuracy_score(y_test, x1))

## Trying random forest using cross validation

In [ ]:
start = datetime.datetime.now()
accuracy = train_model(ensemble.RandomForestClassifier(), train_vectors_dbow, y_train,test_vectors_dbow, y_test)
print ("Radom forest, doc2Vec: ", accuracy)
print(datetime.datetime.now() - start)

## Trying SVM (sigmoid kernal) using cross validation

In [ ]:
# model SVM
start = datetime.datetime.now()
from sklearn.svm import SVC
claffifier1 = SVC(gamma='auto',kernel='sigmoid')
claffifier1.fit(train_vectors_dbow, y_train)
theFinalResult = claffifier1.predict(test_vectors_dbow)
print(datetime.datetime.now() - start)
print('Accuracy SVM',accuracy_score(theFinalResult, y_test)
print('F Score SVM',f1_score(y_test, theFinalResult,average='weighted'))  